# Histopathologic Cancer Detection using Deep Learning

### Importamos las librerias necesarias para el analisis de las imágenes y la realización de las predicciones

In [ ]:
import numpy as np 
import pandas as pd
from glob import glob 
from skimage.io import imread
import os
from pathlib import Path
import keras.backend as K

### Armamos el dataframe con el conjunto de datos de entrenamiento

In [68]:
train_dir = '../iis-project/train'
df = pd.DataFrame({'path': glob(os.path.join(train_dir,'*.tif'))})
labels = pd.read_csv("../iis-project/train_labels.csv")
df['label'] = labels['label']
df.head(10)


,path,label
0,../iis-project/train/052fddb3940ec9f4b9d7d0925...,0
1,../iis-project/train/70e728f78f937dfb16e9867e8...,1
2,../iis-project/train/9b59ba4f1bb5ea63828450fdc...,0
3,../iis-project/train/e0dfb12d1e7116ffa26b7a967...,0
4,../iis-project/train/ea62cbcd62f4438f47902b1bf...,0
5,../iis-project/train/cc5be12926073b97cb9d5146e...,0
6,../iis-project/train/5bf198b5f0732515d0d402a85...,1
7,../iis-project/train/09e441c22952ad49a8a4acb81...,1
8,../iis-project/train/6dfe8a3ca29bff5a2bb108ec2...,0
9,../iis-project/train/7e4b77f241ef3d8d21b188b75...,0


### Se selecciona una muestra de cada etiqueta

           
    Se crean 2 dataframes, uno para cada etiqueta, cada uno de ellos con 10000 elementos ya que tomar en cuenta todo el conjunto no resulta práctico por las limitaciones de recursos fisicos como memoria RAM y/o disco. A partir de estos dataframes se crea uno nuevo que contenga el conjunto de datos reducido.

In [104]:
sample_label0 = df[df.label == 0].sample(10000, random_state=50)
sample_label1 = df[df.label == 1].sample(10000, random_state=50)


reduced_df = pd.DataFrame(pd.concat([sample_label0, sample_label1]))
reduced_df

,path,label
14294,../iis-project/train/49d061af6863059d9afabf7e2...,0
83175,../iis-project/train/26891da24730bc1c924b1ad13...,0
74374,../iis-project/train/39f9bd4975b0b95f840450a91...,0
26984,../iis-project/train/d9b9cd49eb89bb553f88f183a...,0
57505,../iis-project/train/11f6b098f3fb28e0db9698ac8...,0
197135,../iis-project/train/9fd8451bad7342aa3ba77dca8...,0
31167,../iis-project/train/3312e73673436be1ea66f1bc1...,0
77031,../iis-project/train/bbaf303a6a3ac009b0bd91d76...,0
70712,../iis-project/train/a67746c73686d14cf97a99811...,0
26579,../iis-project/train/061007f8eb6746ccc272a8c5f...,0


In [79]:
sample_label0.shape
sample_label1.shape
reduced_df.shape

reduced_df['image'] = reduced_df['path'].map(imread)
reduced_df



,path,label,image
14294,../iis-project/train/49d061af6863059d9afabf7e2...,0,"[[[227, 177, 206], [250, 200, 227], [179, 126,..."
83175,../iis-project/train/26891da24730bc1c924b1ad13...,0,"[[[107, 50, 144], [161, 102, 194], [187, 122, ..."
74374,../iis-project/train/39f9bd4975b0b95f840450a91...,0,"[[[197, 143, 203], [224, 174, 233], [225, 180,..."
26984,../iis-project/train/d9b9cd49eb89bb553f88f183a...,0,"[[[60, 32, 83], [32, 8, 42], [23, 6, 22], [158..."
57505,../iis-project/train/11f6b098f3fb28e0db9698ac8...,0,"[[[239, 217, 229], [221, 199, 211], [253, 229,..."
197135,../iis-project/train/9fd8451bad7342aa3ba77dca8...,0,"[[[66, 47, 92], [25, 0, 36], [215, 179, 205], ..."
31167,../iis-project/train/3312e73673436be1ea66f1bc1...,0,"[[[135, 72, 163], [96, 35, 130], [106, 50, 147..."
77031,../iis-project/train/bbaf303a6a3ac009b0bd91d76...,0,"[[[89, 52, 158], [85, 45, 142], [97, 53, 140],..."
70712,../iis-project/train/a67746c73686d14cf97a99811...,0,"[[[234, 198, 212], [240, 204, 218], [225, 189,..."
26579,../iis-project/train/061007f8eb6746ccc272a8c5f...,0,"[[[237, 189, 201], [222, 173, 194], [174, 124,..."


In [99]:
print(reduced_df['image'][45050])
reduced_df.shape

[[[202 139 166]
  [208 147 178]
  [148  91 124]
  ...
  [164 117 163]
  [173 124 171]
  [ 99  47  95]]

 [[174 118 145]
  [184 129 160]
  [186 134 170]
  ...
  [ 85  39  85]
  [114  64 113]
  [118  68 117]]

 [[251 202 231]
  [185 138 170]
  [188 142 179]
  ...
  [111  65 112]
  [ 60  14  61]
  [161 115 162]]

 ...

 [[186 143 153]
  [233 189 204]
  [153 104 125]
  ...
  [103  51  97]
  [196 145 186]
  [112  62 100]]

 [[238 201 209]
  [238 198 209]
  [199 156 175]
  ...
  [124  68 113]
  [164 112 151]
  [198 146 184]]

 [[255 231 238]
  [219 183 195]
  [184 142 162]
  ...
  [125  69 114]
  [139  84 124]
  [143  91 127]]]


(20000, 3)